# MMI_2024_NLP - Week 1

#Lab 1: Part 2


### Jeremie Mabiala



# Introduction

Before we start, please change the name of the notebook to the following format : **Firstname_LASTNAME_Lab1_B_logistic_regression.ipynb**


In some cells and files you will see code blocks that look like this:

```python
##############################################################################
#                    TODO: Write the equation for a line                     #
##############################################################################
pass
##############################################################################
#                              END OF YOUR CODE                              #
##############################################################################
```

You should replace the `pass` statement with your own code and leave the blocks intact, like this:

```python
##############################################################################
#                    TODO: Write the equation for a line                     #
##############################################################################
y = m * x + b
##############################################################################
#                              END OF YOUR CODE                              #
##############################################################################
```

# (B) Logistic Regression Model

In this second part of the lab, we will implement a language identifier trained on the same data, but using Logistic Regression instead of Naive Bayes.

In [1]:
import io, sys, math
import numpy as np
from collections import defaultdict
from tqdm.notebook import tqdm,trange
from typing import Tuple, List, Dict
import random

This function is used to build the dictionary, or vocabulary, which is a mapping from strings (or words) to integers (or indices). This will allow to build vector representations of documents.

In [2]:
def build_dict(filename:str, threshold:int=1)->Tuple[Dict]:
    """
    Input:
    - filename: the name of the data file.
    - threshold: is the minimum number of times the word has to appear in the data to be added to the vocabulary.
    Output:
    - word_dict: the vocabulary generated from the dataset.
    - label_dict: the dictionary of the labels, with labels as keys and their indices as values of these keys.
    """
    fin = io.open(filename, 'r', encoding='utf-8')
    word_dict, label_dict = {}, {}
    counts = defaultdict(lambda: 0)
    for line in tqdm(fin):
        tokens = line.split()
        label = tokens[0]

        if not label in label_dict:
            label_dict[label] = len(label_dict)

        for w in tokens[1:]:
            counts[w] += 1

    for k, v in counts.items():
        if v > threshold:
            word_dict[k] = len(word_dict)
    return word_dict, label_dict

In [3]:
word_dict, label_dict = build_dict('train1.txt')

0it [00:00, ?it/s]

This function is used to load the training dataset, and build vector representations of the training examples. In particular, a document or sentence is represented as a bag of words. Each example correspond to a sparse vector ` x` of dimension `V`, where `V` is the size of the vocabulary. The element `j` of the vector `x` is the number of times the word `j` appears in the document.

In [4]:
def load_data(filename:str, word_dict:Dict, label_dict:Dict)->List[Tuple]:
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    dim = len(word_dict) #The size of the vocabulary.
    for line in tqdm(fin):
        tokens = line.split() #Consider tokenization by space in this case.
        label = tokens[0]

        yi = label_dict[label]
        xi = np.zeros(dim)
        for word in tokens[1:]:
            if word in word_dict:
                wid = word_dict[word]
                xi[wid] += 1.0
        data.append((yi, xi))
    return data

In [8]:
d = load_data('train1.txt', word_dict=word_dict, label_dict=label_dict)
d[:5]

0it [00:00, ?it/s]

[(0, array([1., 1., 1., ..., 0., 0., 0.])),
 (0, array([0., 0., 0., ..., 0., 0., 0.])),
 (1, array([0., 0., 0., ..., 0., 0., 0.])),
 (0, array([0., 0., 0., ..., 0., 0., 0.])),
 (2, array([0., 0., 0., ..., 0., 0., 0.]))]

First, let's implement the softmax function. Don't forget numerical stability!

In [9]:
def softmax(x:np.ndarray)->np.ndarray:
  ##########################################################################
  #                      TODO: Implement this function                     #
  ##########################################################################
  # Replace "pass" statement with your code
  z = np.sum(np.exp(x-max(x)))

  return np.exp(x-max(x))/z
  ##########################################################################
  #                            END OF YOUR CODE                            #
  ##########################################################################

Now, let's implement the main training loop, by using stochastic gradient descent. The function will iterate over the examples of the training set. For each example, we will first compute the loss, before computing the gradient and performing the update.

In [43]:
def sgd(w:np.ndarray, data:List[Tuple], niter:int, lr:float = 0.001)->np.ndarray:
    """
    Input:
    - w: the weight matrix of shape (length of label dictionary, length of word dictionary)
    - data: the dataset.
    - niter: number of epochs, or number of passes on the all dataset.
    - lr: the learning rate.

    Output:
    - w: the weight matrix.
    """
    random.seed(123)
    nlabels, dim = w.shape
    loss_lis = []

    for iter in trange(niter):
      ##########################################################################
      #                      TODO: Implement this function                     #
      ##########################################################################
      # Replace "pass" statement with your code

      for label, x in data:
         one_hot_label = np.eye(nlabels)[label] #true_labels 

         s= np.dot(w,x)

         ## Softmax out
         out = softmax(s) 

         ## Gradient at w
         dlw = (out-one_hot_label).reshape(-1,1)*x.reshape(-1,1).T

         ## Update rule:
         w = w - lr* dlw
      ##########################################################################
      #                            END OF YOUR CODE                            #
      ##########################################################################

    return w # Replace "..." statement with your code

The next function will predict the most probable label corresponding to example `x`, given the trained classifier `w`.

In [44]:
def predict(w:np.ndarray, x:np.ndarray)->np.ndarray:
  ##########################################################################
  #                      TODO: Implement this function                     #
  ##########################################################################
  # Replace "pass" statement with your code
  return np.argmax(np.dot(w,x))
  ##########################################################################
  #                            END OF YOUR CODE                            #
  ##########################################################################

Finally, this function will compute the accuracy of a trained classifier `w` on a validation set.

In [14]:
def compute_accuracy(w:np.ndarray, valid_data:List[Tuple])->float:
  ##########################################################################
  #                      TODO: Implement this function                     #
  ##########################################################################
  # Replace "pass" statement with your code
  acc = 0.0
  for label, x in valid_data:
    new_label = predict(w,x)
    if new_label == label:
      acc +=1
  return acc/len(valid_data)
  ##########################################################################
  #                            END OF YOUR CODE                            #
  ##########################################################################


In [45]:
print("")
print("** Logistic Regression **")
print("")

word_dict, label_dict = build_dict("train1.txt")
train_data = load_data("train1.txt", word_dict, label_dict)
valid_data = load_data("valid1.txt", word_dict, label_dict)

nlabels = len(label_dict)

dim = len(word_dict)
w = np.zeros([nlabels, dim])
w = sgd(w, train_data, 25)
print("")
print("Validation accuracy: %.3f" % compute_accuracy(w, valid_data))
print("")


** Logistic Regression **



0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]


Validation accuracy: 0.882



# Recommended Reading:

- https://people.tamu.edu/~sji/classes/LR.pdf
